# Replay Data
This notebook allows quick playing with Ascent actions on created "conduit blueprint data". See Ascent's [Extracts](https://ascent.readthedocs.io/en/latest/Actions/Extracts.html) docs for deeper details on Extracts.

In [ ]:
import conduit
import conduit.blueprint
import conduit.relay
import ascent

from IPython.display import Image

import glob

In [ ]:
#print(conduit.about())

In [ ]:
# show existing conduit blueprint files
sim = "./"
glob.glob(sim + "*root")

In [ ]:
# VisIt 2.13 or newer, when built with Conduit support, can visualize meshes from these files.
# Look at the Blueprint HDF5 extract with VisIt
#!visit -o conduit_blueprint.cycle_000400.root

# Prepare Data
Initialize Ascent and Load a Blueprint File

In [ ]:
# this step can take a while if the simulation used high resolution
data = conduit.Node()
conduit.relay.io.blueprint.read_mesh(data, sim + "conduit_blueprint.cycle_000400.root")

In [ ]:
# we publish this data now once to visualize it in multiple ways below
a = ascent.Ascent()
opts = conduit.Node()
opts["actions_file"] = "" # work-around: ascent_actions.yaml file must not overwrite our replay action
#opts["exceptions"] = "forward"
a.open(opts)
a.publish(data)

In [ ]:
def rerender():
    """This is reads an updated Ascent Action and rerenders the image"""
    actions = conduit.Node()
    actions.load("replay_actions.yaml")
    a.execute(actions)

## Relay Example
The next two cells are as follows:
1. modify the Ascent action (examples: https://ascent.readthedocs.io/en/latest/Actions/Examples.html)
2. rerender

By only modifying and executing only those two cells one can quickly iterate over new visualizations.

In [ ]:
%%writefile replay_actions.yaml

# this block are data pipelines
#   each entry in pipelines: executes a series of functions from top to bottom,
#   results of prior functions can be used in later calls of the same pipeline
-
  action: "add_pipelines"
  pipelines:
    slice_field:
      f1:
        type: "slice"
        params:
          topology: topo
          point:  {x: 0.0, y: 0.0, z: 0.0}
          normal: {x: 1.0, y: 1.0, z: 0.0}
                
    sampled_particles:
      f1:
        type: histsampling
        params:
          field: particle_electrons_uz
          bins: 64
          sample_rate: 0.90
      f2:
        type: "clip"
        params:
          topology: particle_electrons # particle data
          multi_plane:
            point1:  {x: 0.0, y:  0.0, z: 0.0}
            normal1: {x: 1.0, y:  0.0, z: 0.0}
            point2:  {x: 0.0, y:  0.0, z: 0.0}
            normal2: {x: 0.7, y: -0.7, z: 0.0}

    clipped_volume:
      f0:
        type: "contour"
        params:
          field: "Ez"
          levels: 10
      f1:
        type: "clip"
        params:
          topology: topo # name of the amr mesh
          multi_plane:
            point1:  {x: 0.0, y:  0.0, z: 0.0}
            normal1: {x: 1.0, y:  0.0, z: 0.0}
            point2:  {x: 0.0, y:  0.0, z: 0.0}
            normal2: {x: 0.3, y: -0.3, z: 0.0}

    mag_clipped_volume_E:
      f0: 
        type: "composite_vector"
        params: 
          field1: "Ex"
          field2: "Ey"
          field3: "Ez"
          output_name: "E_vec"
      f1: 
        type: "vector_magnitude"
        params: 
          field: "E_vec"
          output_name: "E_mag"
      f2:
        type: "contour"
        params:
          field: "E_mag"
          levels: 4
      f3:
        type: "clip"
        params:
          topology: topo # name of the amr mesh
          multi_plane:
            point1:  {x: 0.0, y:  0.0, z: 0.0}
            normal1: {x: 1.0, y:  0.0, z: 0.0}
            point2:  {x: 0.0, y:  0.0, z: 0.0}
            normal2: {x: 0.7, y: -0.7, z: 0.0}

# A scene is describes the things to be rendered and how
#   here one selects either data directly or data that goes into a data pipeline;
#   then, the result gets represented with a visualization "type" and according
#   parameters such as colorbars and color ranges (aka transfer functions)
#   reference:    https://ascent.readthedocs.io/en/latest/Actions/Scenes.html
#   color tables: https://ascent.readthedocs.io/en/latest/Actions/VTKmColorTables.html
-
  action: "add_scenes"
  scenes:
    scene1:
      plots:
#        p0:
#          type: "pseudocolor"
#          field: "particle_electrons_uz"
#          pipeline: "sampled_particles"
        p1:
          type: "pseudocolor"
          field: "Ez"
          pipeline: "clipped_volume"
#          color_table: 
#            name: "plasma"
#            reverse: true
#            control_points: 
#              - 
#                type: "alpha"
#                position: 0.0
#                alpha: 0.5
#              - 
#                type: "alpha"
#                position: 1.0
#                alpha: 0.5
#        p2:
#          type: "pseudocolor"
#          field: "E_mag"
#          pipeline: "mag_clipped_volume_E"
#          color_table: 
#            name: "Black-Body Radiation"
#            reverse: true
        #p3:
        #  type: "pseudocolor"
        #  field: "Ez"
        #  pipeline: "slice_field"
        #  min_value: -3.0e11
        #  max_value: 3.0e11
      renders:
        image1:
          image_width: 512
          image_height: 512
          bg_color: [1.0, 1.0, 1.0]
          fg_color: [0.0, 0.0, 0.0]
          image_prefix: "./replay_%06d"
          camera:
            azimuth: -70
            elevation: 30
            zoom: 1.5

In [ ]:
rerender()
ascent.jupyter.AscentViewer(a).show()

In [ ]:
#!ls *png

In [ ]:
# Image(filename="lwfa_Ex_e-uz_000400.png")
# print(conduit.about())